In [1]:
# https://github.com/ellarauth/CCN-proxy-modeling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
# the measurement station codes (35 stations)
# more detailed information on the stations can be found in metadata/measurement_sites_info.txt
stations = ['ABZ', 'ALE', 'AMA', 'AMM', 'ASP', 'BEI', 'BOT', 'BSL', 'DEL', 'EGB',
            'FKL', 'HAD', 'HEL', 'HPB', 'HRW', 'HYY', 'KCE', 'KPZ', 'MAR', 'MHD', 
            'MLP', 'MUK', 'NAN', 'NEU', 'POV', 'SAO', 'SCH', 'SGP', 'UAE', 'PRL',
            'VAR', 'VHL', 'VIE', 'WAL', 'ZOT']

In [3]:
def combine_datasets(folder):
    '''
    Concatenates the individual datasets for all measurement stations in a specified folder located in "data/".
    
    Parameters:
    folder (str): Name of folder located in the "data/" folder of the repository

    Returns:
    pd.DataFrame: data frame of the concatenated data sets in the folder
    '''
    
    full_df = []

    for s in stations:
        df = pd.read_csv('data/'+folder+'/'+s+'.csv')
        df['station'] = s
        full_df.append(df)

    full_df = pd.concat(full_df)
    full_df = full_df.reset_index(drop=True)
    full_df['id'] = [full_df.station[i] + '-' + str(full_df.date[i]) for i in range(full_df.shape[0])]
    
    return full_df

In [4]:
# loading the aerosol mixing ratio data and saving it as a separate csv file
# a full description of all variable names can be found in metadata/variable_names.txt
aerosols_df = combine_datasets('aerosols')
aerosols_df = aerosols_df[['id', 'latitude', 'longitude', 'aermr01', 'aermr02', 
                           'aermr03', 'aermr04', 'aermr05', 'aermr06', 'aermr07', 
                           'aermr08', 'aermr09', 'aermr10', 'aermr11']]
aerosols_df.to_csv('data/aerosols_data.csv', index=False)

In [5]:
# loading the atmospheric data, calculating the relative humidity, and saving it as a separate csv file
atmospheric_df = combine_datasets('atmospheric')
atmospheric_df = atmospheric_df[['id', 'd2m', 't2m']]
td = atmospheric_df.d2m - 273.15
t = atmospheric_df.t2m - 273.15
atmospheric_df['rh'] = 100*(np.exp((17.625*td) / (243.04+td)) / np.exp((17.625*t) / (243.04+t)))
atmospheric_df.to_csv('data/atmospheric_data.csv', index=False)

In [6]:
# loading the boundary layer height data and saving it as a separate csv file
boundary_layer_height_df = combine_datasets('boundary_layer_height')
boundary_layer_height_df = boundary_layer_height_df[['id', 'blh']]
boundary_layer_height_df.to_csv('data/boundary_layer_height_data.csv', index=False)

In [7]:
# loading the gas data and saving it as a separate csv file
gases_df = combine_datasets('gases')
gases_df = gases_df[['id', 'co', 'c5h8', 'no2', 'no', 'so2']]
gases_df.to_csv('data/gases_data.csv', index=False)

In [8]:
# loading the slow access data and saving it as a separate csv file
slow_access_df = combine_datasets('slow_access')
slow_access_df = slow_access_df[['id', 'nh3', 'crwc', 'c10h16']]
slow_access_df.to_csv('data/slow_access_data.csv', index=False)

In [9]:
# loading the wind data, calculating the wind speed, and saving it as a separate csv file
wind_df = combine_datasets('wind')
wind_df['wind_speed'] = np.sqrt(wind_df.u**2 + wind_df.v**2)
wind_df = wind_df[['id', 'wind_speed']]
wind_df.to_csv('data/wind.csv', index=False)

In [10]:
# loading the n100 data and saving it as a separate csv file
n100_df = []
for s in stations:
    df = pd.read_table('data/N100_proxy/'+s+'_N100.dat', sep='\s+', 
                       names=['year', 'month', 'day', 'hour', 'minute', 'n100'])
    
    # AMA and UAE are in UTC, so the times of the measurements need to be adjusted to local time
    if (s == 'AMA'):
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.date -= timedelta(hours=4)
        df.date = [d.replace(hour=0, minute=0, second=0) for d in df.date]      
    elif (s == 'UAE'):
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.date += timedelta(hours=4)
        df.date = [d.replace(hour=0, minute=0, second=0) for d in df.date]
        
    # the year for FKL is multiplied by 2 due to some bug, so needs to be corrected
    elif (s == 'FKL'):
        df.year /= 2
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])  
        
    else:
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
    
    df.drop(columns=['year', 'month', 'day', 'hour', 'minute'], inplace=True)
    df.set_index('date', inplace=True)
    
    # create empty 3 hourly dataframe - normalize converts time to midnight
    every_3h_index = pd.date_range(df.index.min().normalize(), df.index.max(), freq='3H')
    every_3h_df = pd.DataFrame(index=every_3h_index, columns=['n100'])
    
    # concatenate original and 3 hourly df, drop duplicate timestamps keeping observed values
    df = pd.concat([df, every_3h_df], join='outer').sort_index()
    df = df.loc[~df.index.duplicated(keep='first')]
    
    # interpolate 3 hourly missing (NaN) values, drop edge cases
    df = df.interpolate(method='time').dropna()
    
    df['date'] = df.index
    df['station'] = s
    n100_df.append(df)

n100_df = pd.concat(n100_df)
n100_df = n100_df[n100_df.n100 > 0]
n100_df = n100_df.reset_index(drop=True)
n100_df['id'] = [n100_df.station[i] + '-' + str(n100_df.date[i]) for i in range(n100_df.shape[0])]
n100_df = n100_df[['id', 'station', 'date', 'n100']]
n100_df.to_csv('data/n100_data.csv', index=False)

In [11]:
# combining all the data sets into one dataframe
data = n100_df.merge(aerosols_df, on='id')
data = data.merge(atmospheric_df, on='id')
data = data.merge(boundary_layer_height_df, on='id')
data = data.merge(gases_df, on='id')
data = data.merge(slow_access_df, on='id')
data = data.merge(wind_df, on='id')

# removing outliers for VHL
for i, row in data.loc[data.station == 'VHL'].iterrows():
    if row.n100 > 10000:
        data.drop(index=i, inplace=True)

# reordering the columns
data = data[['id', 'station', 'date', 'latitude', 'longitude', 'n100', 
             'aermr01', 'aermr02', 'aermr03', 'aermr04', 'aermr05', 'aermr06', 'aermr07', 
             'aermr08', 'aermr09', 'aermr10', 'aermr11', 'co', 'c5h8', 'c10h16', 'nh3', 
             'no', 'no2', 'so2', 'd2m', 't2m', 'crwc', 'blh', 'rh', 'wind_speed']]

# saving the full dataset
data.date = pd.to_datetime(data.date)
data = data.dropna(axis=0)
data.to_csv('data/full_data.csv', index=False)
data.head()

,id,station,date,latitude,longitude,n100,aermr01,aermr02,aermr03,aermr04,...,nh3,no,no2,so2,d2m,t2m,crwc,blh,rh,wind_speed
0,ABZ-2012-01-26 18:00:00,ABZ,2012-01-26 18:00:00,50.57,12.99,2860.264407,1.130925e-11,9.656489e-10,5.386068e-11,8.925737e-15,...,7.087745e-10,1.416274e-09,2.257734e-08,5.335189e-09,265.94745,268.02210,0.0,341.34552,85.351851,2.343428
1,ABZ-2012-01-26 21:00:00,ABZ,2012-01-26 21:00:00,50.57,12.99,2765.461017,1.221719e-11,1.043291e-09,4.078474e-11,1.820321e-15,...,9.279780e-10,3.197272e-09,2.466686e-08,7.637095e-09,264.51917,266.25134,0.0,256.22314,87.455067,2.290729
2,ABZ-2012-01-27 00:00:00,ABZ,2012-01-27 00:00:00,50.57,12.99,2073.905085,7.723116e-12,6.594735e-10,1.707053e-11,8.950445e-14,...,1.503275e-09,9.322101e-09,2.733719e-08,1.188354e-08,264.04578,265.29468,0.0,172.99615,90.734149,1.933842
3,ABZ-2012-01-27 03:00:00,ABZ,2012-01-27 03:00:00,50.57,12.99,1711.154237,4.699148e-12,4.011062e-10,8.261608e-12,3.943629e-13,...,1.589063e-09,1.865987e-08,2.964443e-08,1.186046e-08,263.76500,265.05933,0.0,147.33954,90.393698,1.588926
4,ABZ-2012-01-27 06:00:00,ABZ,2012-01-27 06:00:00,50.57,12.99,2081.242373,3.720708e-12,3.175671e-10,4.759155e-12,1.120592e-12,...,1.028960e-09,3.099397e-08,2.678388e-08,7.433328e-09,264.02032,265.33960,0.0,112.06348,90.239081,1.724053
